# Understanding the data
There is a lot of information in these files, most of which I don't understand.  Can we figure out which rows are associated with each proposal?  Can we determine the status of the proposal?
-Tristan

# Setup

In [1]:
# packages
import datetime
import matplotlib as mp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

# seaborn settings
sns.set()
sns.set_style('whitegrid')
sns.set_style('ticks')

In [2]:
# load data
fields = pd.read_csv('planning-department-records-2018/DataSF_PPTS_Fields.csv')
data = pd.read_csv('planning-department-records-2018/PPTS_Records_data.csv',
                   parse_dates=['date_opened','date_closed',],
                   infer_datetime_format=True)

C:\Users\Tristan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (28,31,32,33,34,36,38,39,40,41,42,43,44,45,46,48,119,163,167,168,169) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Record types
Through some sleuthing, I've determined that there is a hierarchy of categorization.

- record_type_group: 'planning' for all data
- record_type_type: 5 categories: 'Project', 'Applications', 'Complaint', 'Other', 'Research'
- record_type_subtype: 17 categories
- record_type: 63 categories.  Each one, except for 'Other' is given a 3-letter abbreviation.  The abbreviations are in the column record_type_category.  Note that there are a bunch of typos in these two columns, mostly when people fail to follow a consistent format.
- record_type_4level contains the entire hierarchy

In [82]:
sorted(data['record_type_4level'].unique())

['Planning/Applications/Appeal/APL',
 'Planning/Applications/Citywide Planning/CWP',
 'Planning/Applications/Citywide Planning/GPL',
 'Planning/Applications/Commission Review/CRV',
 'Planning/Applications/Commission Review/IMP',
 'Planning/Applications/Discretionary Review/DRM',
 'Planning/Applications/Discretionary Review/DRP',
 'Planning/Applications/Environmental/CATEX-EEC',
 'Planning/Applications/Environmental/Community Plan Determin-ECD',
 'Planning/Applications/Environmental/Community Plan Exemption-ECE',
 'Planning/Applications/Environmental/Community Plan Focused EIR-EIF',
 'Planning/Applications/Environmental/EIR Addendum-EIA',
 'Planning/Applications/Environmental/EIR Fee',
 'Planning/Applications/Environmental/ENV',
 'Planning/Applications/Environmental/Environmental T and M-ETM',
 'Planning/Applications/Environmental/Initial Study-EEA',
 'Planning/Applications/Environmental/Neg Dec Addendum-ENA',
 'Planning/Applications/Environmental/Transportation Abbrev-ETA',
 'Planning/

In [85]:
# make a dict expanding out each abbreviation.
# also, print out any anomalies in the record_type column.

# note that there are also anomalies in the record_type_category column
# it seems it's supposed to contain 3-letter abbreviations, but the environmental department doesn't follow that format.

oddity_list = []
record_abbr = {}
for record_type in data['record_type'].unique():
    m = re.match('^(.*) \((...)\)',record_type)
    if(m):
        if m.groups()[1] in record_abbr:
            oddity_list.append(record_type)
        else:
            record_abbr[m.groups()[1]] = m.groups()[0]
    else:
        oddity_list.append(record_type)

record_type_dict['Other'] = 'Other'

print(oddity_list)

['Environmental', 'Conditional Use Authorization', 'Variance', 'General Plan Referral', 'Discretionary Review - Public Initiated', 'Other', 'Discretionary Review - Manditory (DRM)']


# Families of associated entries
Each proposal should be associated with several entries.  For instance, there may be one for the project proposal, one for the research, several for applications, etc.  Here I try to group entries into families.

The first method I tried is to look at parents or children.  It seems that this is not always reliable.  Another method is to look at addresses, but this is also not reliable because sometimes the address is nan, or it appears in different formats.  A more reliable method is to use the_geom, which is never nan.  However, the_geom picks out buildings, not projects.  A single building may have a lot of unrelated paperwork associated with it.

In [3]:
#return a list of rows of family members, based on parent/children columns
def get_family(data,row):
    members = {}
    find_family(data,members,row,1)
    return list(members.values())

def find_family(data,members,curr_row,stacklevel):
    if stacklevel>40:
        print('Error: too many recursions on row %i' % curr_row)
        return
    members[data.loc[curr_row,'record_id']] = curr_row
    family = []
    if not pd.isna(data.loc[curr_row,'parent']):
        family += data.loc[curr_row,'parent'].split(',')
    if not pd.isna(data.loc[curr_row,'children']):
        family += data.loc[curr_row,'children'].split(',')
    for member in family:
        if member not in members:
            #find row of the member
            member_match = np.where(data['record_id'] == member)[0]
            if(len(member_match > 0)):
                find_family(data,members,member_match[0],stacklevel+1)


In [104]:
#return a list of rows associated with the same building
def get_building(data,row):
    my_geom = data.loc[row,'the_geom']
    member_match = np.where(data['the_geom'] == my_geom)[0]
    return list(member_match)

#return a list of rows associated with the same address
#unfortunately, addresses aren't always in a consistent format
def get_address(data,row):
    my_address = data.loc[row,'address']
    member_match = np.where(data['address'] == my_address)[0]
    return list(member_match)

In [121]:
#takes the output of get_building, and divides the list into families
#returns a dict with each index as a key, and the family number as the value
def divide_by_family(data,building):
    if type(building) != list:
        building = get_building(data,building)
    family_ids = {}
    i = 1
    for b in building:
        if b not in family_ids:
            family = get_family(data,b)
            for member in family:
                family_ids[member] = i
            i += 1
    return(family_ids)

# Building stories
I thought I'd look at a few examples of buildings, to get a sense of what they look like.

Overall, my impression is that there are a lot of unrelated stories going on with each building.  Sometimes there are multiple project proposals, sometimes there are a bunch of miscellaneous stuff like complaints.  I'm still not sure how to determine the status of a project, but if the information is spread across multiple rows it may be difficult to figure out which rows to look at.  No, the 'family' doesn't help, I checked.

In [126]:
def tell_story(data,row):
    building = get_building(data,row)
    location = data.loc[building,:]
    family_ids = divide_by_family(data,building)
    #print the first valid address
    for i in range(location.shape[0]):
        if not pd.isna(location['address'].iloc[i]):
            print(location['address'].iloc[i])
            break
    location.sort_values('date_opened',inplace=True)
    for i in range(location.shape[0]):
        print('')
        #print('family %s' % family_ids[location.index[i]],location['date_opened'].iloc[i], location['record_type'].iloc[i], location['record_status'].iloc[i])
        print(location['date_opened'].iloc[i], location['record_type'].iloc[i], location['record_status'].iloc[i])
        print(location['description'].iloc[i])

In [68]:
i = 82
print(get_building(data,i))

[82, 5861, 5864, 9766, 18982, 28271, 49426, 89982, 98653, 108095, 138800, 145264, 150667, 152836, 163170, 169650, 178576, 183313]


In [6]:
#we can see that even though it's just one building, the address is inconsistent.
data.loc[get_building(data,i),'address']

82                                        234 01ST ST 94105
5861                                      246 01ST ST 94105
5864                                      234 01ST ST 94105
9766                                      234 01ST ST 94105
18982                  234 01ST ST, SAN FRANCISCO, CA 94105
28271      246 01ST ST, #3RD FLOOR, SAN FRANCISCO, CA 94105
49426                  234 01ST ST, SAN FRANCISCO, CA 94105
89982                  234 01ST ST, SAN FRANCISCO, CA 94105
98653                                     234 01ST ST 94105
108095                                    246 01ST ST 94105
138800                                    246 01ST ST 94105
145264                                    246 01ST ST 94105
150667                                    246 01ST ST 94105
152836                                    246 01ST ST 94105
163170    246 01ST ST, #STREET LEV, SAN FRANCISCO, CA 94105
169650                                    246 01ST ST 94105
178576                                  

In [127]:
#I'm commenting out the part where I print the story, to reduce clutter
#tell_story(data,82)

234 1st St is an apartment building (which also has the address 246 1st St).  In 2006 there was a permit to create a massage establishment on the ground floor.  In 2008, there was a permit to move a tenant on the third floor.  There are a bunch of permits and project proposals related to AT&T and T-mobile wanting to replace equipment at the top of the building.  In 2015, there was a proposal to renovate the building.  It starts out as a PRV (project review meeting), followed by a PRJ (project profile).  A year later they file a PTA (permit to alter), which is approved.  However, the status of the project is marked as "under review" which might mean they still haven't gotten permission.

In 2017, there are a few extra entries.  There are two letters: one TDU (notice of use), and one ZAV (zoning verification).  And it seems like someone applied for it to be designated as a historical landmark, which was approved.

TL;DR: there are a lot of things going on in a single building.  There was one major construction proposal, which I don't think ever got through.  And I wonder if opponents of the proposal made the building into a landmark.

In [70]:
#tell_story(data,3)

900 Sansome is a public parking lot.  In 2009 there was one project to replace a window, which had to get through COA and CUA, but is apparently closed, so maybe it passed?  Most of the rest seems to be an annual renewal of some permit for parking facilities.  There are also a few complaints about signs being wrong

In [71]:
#tell_story(data,0)

875 Sansome is currently a two-story office space.  Since 2016 they have been looking into replacing it with a 6-story mixed use building.  A project profile was finally made in 2018, and it would add 6 residential units (which I take it means that all of them are market rate).  The CUA is currently under review.

In [95]:
#tell_story(data,23499)

383 King St is a condo building.  The project to construct it was started in 2006, and I presume that it finished.  Later, in 2016 and later, there are several complaints about illegal short-term rentals through AirBnB.

# Project statuses
Here I try to determine what kinds of statuses a project can have.  In progress.

In [98]:
data.loc[data['record_type_category']=='PRJ','record_status'].unique()

array(['Accepted', 'Withdrawn', 'Closed', 'Closed - Withdrawn', 'Open',
       'Under Review', 'Application Accepted', nan, 'Complete',
       'Submitted', 'Pending', 'Closed - Approved', 'Approved', 'On Hold',
       'Incomplete', 'Actions Pending', 'Pending Review', 'Denied',
       'Disapproved', 'Closed - Complete'], dtype=object)